In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Define a function for taking an integral

In [ ]:
def func(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return a*x**2 + b*x + c

## Define it's integral so we know the right answer

In [ ]:
def func_integral(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return (a*x**3)/3. + (b*x**2)/2. + c*x

## Define the core of the trapezoid method

In [ ]:
def trapezoid_core(f,x,h):
    return 0.5*h*(f(x+h) + f(x))

## Define a wrapper function to perform trapezoid method

In [ ]:
def trapezoid_method(f,a,b,N):
    # f == function to integrate
    # a == lower limit of integration
    # b == upper limit of integration
    # N == number of functoin evaluations to use
    # note the number of chunks will be N-1
    # so if N is odd, then we don't need to adjust the 
    # last segment
    
    # define x values to perform trapezoid rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    # define the value of the integral
    Fint = 0.0
    
    # perform the integral using the trapezoid method
    for i in range(0,len(x)-1,1):    # giving it range from 0 to n-2 inclusive, in 1 spaces
        Fint += trapezoid_core(f,x[i],h)    # use f and x[i] in trapezoid core
        # print(i,i+1,x[i],x[i]+h,x[-2],x[-1])
        
    # return the answer
    return Fint


## Define the core of the Simpson's Method

In [ ]:
def simpson_core(f,x,h):   # little h is width of each of th little chunks
    return h*( f(x) + 4*f(x+h) + f(x+2*h))/3.

## Define a wrapper function to perform Simpson's Method

In [ ]:
def simpsons_method(f,a,b,N):
    # f == function to integrate
    # a == lower limit of integration
    # b == upper limit of integration
    # N == number of functoin evaluations to use
    # note the number of chunks will be N-1 intervals b/w those function evaluations
    # so if N is odd, then we don't need to adjust the 
    # last segment
    
    # define x values to perform simpsons rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    # define the value of the integral
    Fint = 0.0
    
    # perform the integral using simpson's method
    for i in range(0, len(x)-2,2):       # goint from 0 to n-2 means from 0 to n-3 inclusive
        Fint += simpson_core(f,x[i],h)   # in two chunk steps

       
    # apply simpson's rule over the last interval if N is even
    if((N%2)==0):
        Fint += simpson_core(f, x[-2],0.5*h)
    
        # print(i,i+2,x[i],x[i]+h,x[i]+2*h,x[-2],x[-1])
        
    return Fint

## Define the Romberg Integration core

In [ ]:
def romberg_core(f,a,b,i):
    
    # we need the difference b-a, entire width of interval we are integarting
    h = b-a
    
    # and the increment between new func evals, dh - delta h
    dh = h/2.**(i)
    
    # we need the cofactor, a number that apperas in front of the other terms(fraction)
    K = h/2.**(i+1)
    
    # and the function evaluations 
    M = 0.0
    for j in range(2**i):           # from zero 2^i
        M += f(a + 0.5*dh + j*dh)   # dh is the offset between midpoint functions, half the interval is the offset 
        
    # return the answer
    return K*M

## Define a wrapper function to perform Romberg Integration

In [ ]:
# f is function
# a,b, are limits of integration 
# tol is tolerance

def romberg_integration(f,a,b,tol):
    
    # define an integration variable
    i = 0
    
    # define a maximum number of iterations
    imax = 1000
    
    # define an error estimate, set to a large value
    # some error delta, takes care of case if someone passes wrong value, it allows
    # function to start iterating
    delta = 100.0 * np.fabs(tol)
    
    # set a narray of integral answers because we need all the successive answers we
    # got prior to i^th iteration
    I = np.zeros(imax,dtype=float)
    
    # get the zero'th romberg iteration
    I[0] = 0.5*(b-a)*(f(a) + f(b))
    
    # I is set to 0, so iterate by 1
    i += 1
    
    # iterate until delta is less than tolerance
    while(delta>tol):
        
        # find this romber iteration
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,i)
        
        # compute the new fractional error estimate
        delta = np.fabs( (I[i]-I[i-1])/I[i]) 
        
        print(i, I[i], I[i-1], delta)
        
        if(delta>tol):
            #iterate
            i+=1
            
            # if we've reached the maximum iterations
            if (i>imax):
                print("Max iterations reached.")
                raise StopIteration('Stopping iterations after ',i)
                
    # return the answer
    return I[i]



## Find out the answer

In [ ]:
# we took the indefinite integral evaluated at 1 and 0
Answer = func_integral(1)-func_integral(0)
print(Answer)

print("Trapezoid")
print(trapezoid_method(func,0,1,10))   # integral from 0 to 1 in steps 10

print("Simson's Method")
print(simpsons_method(func,0,1,10))

print("Romberg")
tolerance = 1.0e-6
RI = romberg_integration(func,0,1,tolerance)
print(RI, (RI-Answer)/Answer, tolerance) 